In [2]:
import numpy as np 
import math

# 1.

The process of active customers is still a Poisson process as a thining process. It's new intensity is $ \lambda p dx $ 

# 2.

If we consider that the cell is the the circle of Radius R, there are, in mean, $ \lambda p \pi R^2 $ active customers.

# 3. 


Instinctively, one can say that the farest we are from the center of the cell the more we need subcarriers to communicate at rate C. Let's compute to see if it is the case. 

Let's take k in $ {1,...,N_{max}} $ and $ x  $ such that :
$$ N(x) = \lceil \frac{C}{W \log_2\left(1+\frac{K}{{\lVert x \rVert}^\gamma}\right)} \rceil = k $$
It is equivalent to : 
$$ k-1 \lt \frac {C}{W \log_2\left(1+\frac{K}{{\lVert x \rVert}^\gamma}\right)} \leq k $$
Which is after calculus equivalent to  : 
$$\left(\frac{K}{\exp\left(\frac{C\ln(2)}{(k-1)W}\right)-1}\right)^{\frac{1}{\gamma}} \leq \lVert x \rVert \lt \left(\frac{K}{\exp\left(\frac{C\ln(2)}{kW}\right)-1}\right)^{\frac{1}{\gamma}}$$
Therefore the geometrical characteriation of the subset of the cell in which all customers require k subcarriers is a ring bounded by the circle of radius $R1=\left(\frac{K}{\exp\left(\frac{C\ln(2)}{kW}\right)-1}\right)^{\frac{1}{\gamma}}$  and the circle of radius $R2=\left(\frac{K}{\exp\left(\frac{C\ln(2)}{(k-1)W}\right)-1}\right)^{\frac{1}{\gamma}}$

# 4.

We need to compute the lebesgue measure of the ring for k=3.

We just need to substract the area of the diferent circles  : 
 $$ lebesgue(ring_3) = \pi(R1^2-R2^2)$$
Then the distribution of A3 is a poisson randomn variable of parametre $ \lambda p*lebesgue(ring_3) $ 

# 5. 

We can simulate the active customer on the all disque of radius R, then draw the ring for k=3 count the number of peapole in the ring and stock this in an array. We repeat this operation to have many sample of $A_3$. Then we check whith kolmogorov smirnov test if the samples follow the law of a poisson randomn varaible of parametre $ \lambda p*lebesgue(ring_3) $ 

# 6. 

We partition the  disk of radius R in the rings for k in $ {1,...,N_{max}} $
Therefore we have : 
$$ F(\phi)= \sum_{x \in \phi} N(x)= \sum_{k=1}^{N_{max}} \sum_{x \in ring_k} N(x) $$
But  if $ x \in ring_k $ then $ N(x)= k $

Thus : 
$$ F(\phi)= \sum_{k=1}^{N_{max}} \sum_{x \in ring_k}k = \sum_{k=1}^{N_{max}} k\xi _k  $$
Where $ \xi_k $ are  Poisson randomn variable with prametre equals to $ \lambda p*lebesgue(ring_k) $ 
They are independent because the different rings are disjoint. 

# 7. 

When $ X_{\mu} $  is Poisson r.v. of parametre $\mu$ we have via tranfert : 
$$\mathbb{E}[\exp(\theta X_{\mu})]=\sum_{k=0}^{\infty} \exp(\theta k)\exp(-\mu)\frac{\mu ^k}{k!}$$
Which is equal after calculus to : 
$$\mathbb{E}[\exp(\theta X_{\mu})]=\exp(\mu(\exp(\theta)-1))$$
Then :
$$ f(\theta)=\exp(-k\mu \theta)\mathbb{E}[\exp(\theta X_{\mu})]=\exp(\mu(\exp(\theta)-1-K\theta))$$
And : 
$$ \frac{\partial f}{\partial \theta}=\mu \exp(\mu(\exp(\theta)-1-K\theta))(\exp(\theta)-K)$$
$$\frac{\partial f}{\partial \theta}=0 \implies \theta=\ln(K)$$
Thus : $$ \min_{\theta \gt 0}\{f(\theta)\}=\exp(\mu(K-1-K\ln(K)))$$

# 8.

By Markov inequality we have : 
$\mathbb{P}(X_{\mu}\geq K\mu) \leq \exp(-K\mu\theta)\mathbb{E}[\exp(\theta X_{\mu})]$

And whith Q7 : 
$\mathbb{P}(X_{\mu}\geq K\mu) \leq \exp(\mu(K-1-K\ln(K)))$

i.e we need to find $K_{\mu}$ such that $K_{\mu}-1-K_{\mu}\ln(K_{\mu}) \leq \frac{1}{\mu}\ln(10^{-4})$

In [3]:
def K_mu(mu): ## a refaire 
    k=0
    while (k-1-k*np.log(k) > (1/mu)*np.log(10**(-4))):
        k+=1
    return k

K_mu(10**(-6))

/var/folders/dc/2vgy793x7t15h4j_f_cy_88h0000gr/T/ipykernel_80315/2707124393.py:3: RuntimeWarning: divide by zero encountered in log
  while (k-1-k*np.log(k) > (1/mu)*np.log(10**(-4))):
/var/folders/dc/2vgy793x7t15h4j_f_cy_88h0000gr/T/ipykernel_80315/2707124393.py:3: RuntimeWarning: invalid value encountered in double_scalars
  while (k-1-k*np.log(k) > (1/mu)*np.log(10**(-4))):


0

# 9. 

In [4]:
#we use the parameters given to us
lam=0.01
R=300
p=0.01
C=250
W=250
K=10**6
gamma=2.8
SNR_min=10**(-1)


N_max= math.ceil(C/W*np.log2(1+SNR_min))
N_max

1

In [5]:
def ring_radius(k,K,C,W,gamma):
    """return the min and max radius of Ringk as defined above"""
    R1=(K/(np.exp(C*np.log(2)/((k-1)*W))-1))**(1/gamma)
    R2=(K/(np.exp(C*np.log(2)/(k*W))-1))**(1/gamma)
    return R1,R2

## Computation of $\mathbb{E}[\xi_k]$ for $k = 1,\dots,N_{max}$

We remind that  

In [6]:

mean_xi_k=[]
mean_xi_k.append(lam*p*np.pi*((K/(np.exp(C*np.log(2)/(1*W))-1))**(1/gamma)**2))
for k in range(2,N_max+1):
    R1,R2=ring_radius(k,K,C,W,gamma)
    mean_xi_k.append(lam*p*np.pi*(R2**2-R1**2))


In [7]:
res=K_mu(mean_xi_k[0])
for m in mean_xi_k: 
    if K_mu(m)>res:
        res=K_mu(m)
print(res)


0


/var/folders/dc/2vgy793x7t15h4j_f_cy_88h0000gr/T/ipykernel_80315/2707124393.py:3: RuntimeWarning: divide by zero encountered in log
  while (k-1-k*np.log(k) > (1/mu)*np.log(10**(-4))):
/var/folders/dc/2vgy793x7t15h4j_f_cy_88h0000gr/T/ipykernel_80315/2707124393.py:3: RuntimeWarning: invalid value encountered in double_scalars
  while (k-1-k*np.log(k) > (1/mu)*np.log(10**(-4))):


In [8]:
### vecotr represnting the xi_k
Matrix_xi_k=np.zeros((N_max,res))
for i in range(N_max):
    for j in range(res):
        mu=mean_xi_k[i]
        Matrix_xi_k[i,j]=np.exp(-mu)*(mu**j)/math.factorial(j)


In [9]:
### vecotr represnting the k*xi_k
Matrix_k_xi_k=np.zeros((N_max,res))
for i in range(N_max):
    for j in range(res):
        if j%k==0:
            Matrix_k_xi_k[i,j]=Matrix_xi_k[i,j]


In [10]:
### distribution of F 

def F(n):
    res=np.convolve(Matrix_k_xi_k[0,:],Matrix_k_xi_k[1,:])
    for i in range(1,N_max-1):
        res=np.convolve(res,Matrix_k_xi_k[i+1,:])
    return res[n]


# 10.

In [11]:
def Find_S(alpha):
    S=0
    while F(S)<alpha:
        S+=1
    return S